# Analyzing models

**cameo** uses and extends the model data structures defined by [cobrapy](https://opencobra.github.io/cobrapy/), our favorite **CO**nstraints-**B**ased **R**econstruction and **A**nalysis tool for **Py**thon. **cameo** is thus 100% compatible with **cobrapy**. For efficiency reasons though **cameo** implements its own analysis methods that take advantage of a more advanced solver interface.

In [1]:
from cameo import models
model = models.bigg.e_coli_core

## Flux Variability Analysis

Flux variability analysis (FVA) enables the computation of lower and upper bounds of reaction fluxes.

In [2]:
from cameo import flux_variability_analysis

In [3]:
fva_result = flux_variability_analysis(model)
fva_result.data_frame

,lower_bound,upper_bound
ACALD,-2.000000e+01,0.000000e+00
ACALDt,-2.000000e+01,-3.155444e-30
ACKr,-2.000000e+01,6.310887e-30
ACONTa,1.294225e-30,2.000000e+01
ACONTb,0.000000e+00,2.000000e+01
ACt2r,-2.000000e+01,0.000000e+00
ADK1,0.000000e+00,1.666100e+02
AKGDH,0.000000e+00,2.000000e+01
AKGt2r,-1.000000e+01,2.110203e-29
ALCD2x,-2.000000e+01,7.888609e-30


In [4]:
fva_result.plot(index=fva_result.data_frame.index[:25])

One very useful application of FVA is determining if alternative optimal solution exist.

In [5]:
fva_result2 = flux_variability_analysis(model,fraction_of_optimum=0.5)
fva_result2.data_frame

,lower_bound,upper_bound
ACALD,-1.260245e+01,0.000000e+00
ACALDt,-1.260245e+01,-5.048710e-29
ACKr,-1.335885e+01,0.000000e+00
ACONTa,4.714370e-01,1.383029e+01
ACONTb,4.714370e-01,1.383029e+01
ACt2r,-1.335885e+01,-8.835242e-29
ADK1,0.000000e+00,8.433833e+01
AKGDH,0.000000e+00,1.335885e+01
AKGt2r,-6.679426e+00,1.183291e-29
ALCD2x,-1.107161e+01,0.000000e+00


In [6]:
fva_result2.plot()

In [7]:
from cameo.visualization import plotting

Phenotpic Phase Plane
--------------------

The phenotypic phase plane is a modeling technique was developed to do a theoretical accessement of what cell can or cannot do in terms of the stoichiometric constraints [Edawards *et al.* 2001]. 

The phenotipic phase plane between growth and a product of interest yields the production envelope: a representation between the trade of between production of the desired product and growth.

In [8]:
from cameo import phenotypic_phase_plane

In [9]:
model.reactions.EX_o2_e.lower_bound = -10
result = phenotypic_phase_plane(model, 
                                variables=[model.reactions.BIOMASS_Ecoli_core_w_GAM],
                                objective=model.reactions.EX_succ_e,
                                points=10)

In [10]:
result.plot(width=700)

The production envelope allows is a quick way to inspect the limitations of the system to design and how the production relates for with growth. In the previous example, succinate prudction is completly decoupled from growth and by decreasing the growth rate it is theoretically possible to produce up to 15 times more succinate. 

In [11]:
result.plot(height=350, points=[(0.52, 0), (0.23, 12.2)], points_colors=["green", "red"])

The production envelope can show the coupling between growth and production. There is no stoichiometric couple between growth and production for succinate under aerobic conditions, but that is not the case for acetate under anaerobic conditions.

In [12]:
result = phenotypic_phase_plane(model, 
                                variables=[model.reactions.BIOMASS_Ecoli_core_w_GAM],
                                objective=model.reactions.EX_ac_e)
result.plot(height=350)

In [13]:
result.data_frame

,BIOMASS_Ecoli_core_w_GAM,objective_lower_bound,objective_upper_bound
0,0.000000,0.00000,20.000000
1,0.029424,0.00000,19.552804
2,0.058847,0.00000,19.105607
3,0.088271,0.00000,18.658411
4,0.117695,0.00000,18.211215
5,0.147119,0.00000,17.764018
6,0.176542,0.00000,17.316822
7,0.205966,0.00000,16.869626
8,0.235390,0.00000,16.422429
9,0.264813,0.00000,15.975233


One can imediatly see if a design is feasible within the new defined constraints.

In [14]:
result.plot(height=350, points=[(0.2, 8), (0.2, 2)], points_colors=["green", "red"])

The computed data can be inspected in the format of a pandas data frame by calling `result.data_frame`

In [15]:
result.data_frame

,BIOMASS_Ecoli_core_w_GAM,objective_lower_bound,objective_upper_bound
0,0.000000,0.00000,20.000000
1,0.029424,0.00000,19.552804
2,0.058847,0.00000,19.105607
3,0.088271,0.00000,18.658411
4,0.117695,0.00000,18.211215
5,0.147119,0.00000,17.764018
6,0.176542,0.00000,17.316822
7,0.205966,0.00000,16.869626
8,0.235390,0.00000,16.422429
9,0.264813,0.00000,15.975233


In [16]:
model.reactions.EX_o2_e.lower_bound = 0
result2 = phenotypic_phase_plane(model, 
                                 variables=[model.reactions.BIOMASS_Ecoli_core_w_GAM],
                                 objective=model.reactions.EX_ac_e,
                                 points=10)
result2.plot()

## Flux Balance Impact Degree

In [17]:
from cameo.flux_analysis.analysis import flux_balance_impact_degree

In [18]:
model.reactions.EX_o2_e.lower_bound = -10

In [19]:
%time fbid = flux_balance_impact_degree(model, ["EX_o2_e"])

CPU times: user 213 ms, sys: 3.53 ms, total: 217 ms
Wall time: 264 ms


In [20]:
fbid

,perturbed
ACALD,False
ACALDt,False
ACKr,False
ACt2r,False
ADK1,False
ALCD2x,False
ATPM,False
ATPS4r,True
CO2t,False
CYTBD,True
